In [147]:
import numpy
from scipy.stats import lognorm
from pt.plotter import *
from bqplot import LogScale

days_per_month = 365.25/12.0
minutes_per_day = 60.0*24.0
minutes_per_month = days_per_month * minutes_per_day

size_per_datastore_span = 1000
cost_per_storage_byte = 0.5 / 1000000000

transactions_per_minute = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]
set_default_plotter_options({
    'lambda_span_bias': [ float("%4.2f" % (0.05 * x)) for x in range(1, 40)],
    'size_per_span': [100, 200, 300, 400],
    'transactions_per_minute': transactions_per_minute,
    'refresh_rate': [0.25, 0.5, 1, 2, 5, 10],
    'percent_datastore_spans': [5*x for x in range(21)],
    'size_per_span': [100, 150, 250, 350, 450],
    'text_compression_ratio': [x/10.0 for x in range(1, 11)]
})

In [148]:
xdata = numpy.logspace(0, numpy.log10(1000), 100)

span_count = lognorm(1, scale=14.33, loc=-1)

def gen_ln(x, lambda_span_bias=1.0):
    return span_count.cdf((x-1) / lambda_span_bias)

spans = make_quick_plot(
        xdata, gen_ln,
        hline(0.37), # X < 10
        hline(0.67), # X < 20
        hline(0.90), # X < 50
        hline(0.97), # X < 100
        title='Spans Per Transaction',
        ax='Span Event Count Per Transaction',
        ay='Cumulative Probability',
        xscale=LogScale,
        defaults={
            'lambda_span_bias': 1.0
        })


Figure(axes=[Axis(label='Span Event Count Per Transaction', scale=LogScale(max=1000.0, min=1.0)), Axis(label='…

In [149]:
def gen_cost_every_transaction(xdata):
    return [x*minutes_per_month*(0.01/1000) for x in xdata]

spans = make_quick_plot(
        transactions_per_minute, gen_cost_every_transaction,
        title='Cloudwatch-API-query cost, when querying invocations/minute for every transaction',
        ax='Transactions Per Minute',
        ay='Cost Per Month ($)',
        xscale=LogScale,
        defaults={})


VBox()

Figure(axes=[Axis(label='Transactions Per Minute', scale=LogScale(max=10000.0, min=1.0)), Axis(label='Cost Per…

In [150]:
concurrent_executions = [1, 2, 5, 10, 20]

def gen_cost_refresh(xdata, refresh_rate=1.0):
    return [minutes_per_month*x*(0.01/1000)/refresh_rate for x in xdata]

spans = make_quick_plot(
        concurrent_executions, gen_cost_refresh,
        title='Cloudwatch-API-query cost when updating each execution once/minute',
        ax='Average Concurrent Executions',
        ay='Cost Per Month ($)',
        ysc_min=0.0,
        ysc_max=40.0,
        defaults={
            'refresh_rate': 1.0
        })


Figure(axes=[Axis(label='Average Concurrent Executions', scale=LinearScale(max=20.0, min=1.0)), Axis(label='Co…

In [151]:
hub_refresh_rate = [1.0/x for x in [1, 2, 5, 10, 20]]

def gen_cost_every_minute(xdata):
    return [minutes_per_month*(0.01/1000)*x for x in xdata]

spans = make_quick_plot(
        hub_refresh_rate, gen_cost_every_minute,
        title='Cloudwatch-API-Query cost when updating each execution once/minute',
        ax='Average Concurrent Executions',
        ay='Cost Per Month ($)',
        ysc_min=0.0,
        ysc_max=1.0,
        defaults={})


VBox()

Figure(axes=[Axis(label='Average Concurrent Executions', scale=LinearScale(max=1.0, min=0.05)), Axis(label='Co…

In [158]:
spans_per_transaction_percentile = range(1, 101)


# TODO: ADD DATASTORE SPANS
def gen_log_cost(xdata, transactions_per_minute=1,
                 lambda_span_bias=1.0, percent_datastore_spans=0.0,
                 size_per_span=350, text_compression_ratio=1.0):

    fraction_datastore_spans = percent_datastore_spans / 100
    return [
        (minutes_per_month*transactions_per_minute*text_compression_ratio*
         cost_per_storage_byte*lambda_span_bias*span_count.ppf(x/100)*(
                size_per_span*(1-fraction_datastore_spans)
              + (size_per_span+size_per_datastore_span)*lambda_span_bias*x*fraction_datastore_spans))
            for x in xdata]

spans = make_quick_plot(
        spans_per_transaction_percentile, gen_log_cost,
        title='Cloudwatch-log storage cost for spans',
        ax='Percentile, spans per transaction',
        ay='Cost Per Month ($)',
        ysc_min=0.0,
        ysc_max=100.0,
        defaults={
            'lambda_span_bias': 1.0,
            'transactions_per_minute': 1000,
            'percent_datastore_spans': 0.0,
            'size_per_span': 350,
            'text_compression_ratio': 1.0
        })

Figure(axes=[Axis(label='Percentile, spans per transaction', scale=LinearScale(max=100.0, min=1.0)), Axis(labe…

In [153]:
span_count.ppf(0.5)

13.33